In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import path
import json
import torch
import lightning.pytorch as pl
from torch.utils.data import DataLoader
import os
from argparse import Namespace
import datetime

import util as tu
import model as tm
from setup import *


c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\TanJI\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
configs = Namespace (
    project_name = "TanJI",
    test_database_ids = [13],
    random_seed = 114514,
    batch_size = 1000,
    accelerator = 'cpu',
    progress_bar = True,
    lr = 0.001,
    dropout = 0.2,
    epochs = 100,
    pad_length = 20,
    node_length = 19,
    hidden_dim = 192,
    output_dim = 1,
    newpath = './results/full/cost/',
    to_predict = 'cost',
    dataset_file_name = 'alldataset.pkl',

    test_data_name = 'accidents'
)

In [4]:
with open(path.join(ROOT_DIR, 'data', 'workload1', 'statistics.json')) as file:
    feature_statistics = json.load(file)
# with open(path.join(ROOT_DIR, 'data', 'imdb', 'statistics.json')) as file:
#     feature_statistics = json.load(file)
configs.node_length = len(feature_statistics['node_types']['value_dict']) + 2
encoder = tu.Encoder(feature_statistics, configs)

{'Index Only Scan': 0, 'Index Scan': 1, 'Seq Scan': 2, 'Hash Join': 3, 'Bitmap Index Scan': 4, 'Result': 5, 'Merge Join': 6, 'Limit': 7, 'Aggregate': 8, 'Sort': 9, 'Hash': 10, 'Bitmap Heap Scan': 11, 'Nested Loop': 12, 'Materialize': 13, 'BitmapOr': 14, 'Memoize': 15, 'Gather': 16, 'Gather Merge': 17, 'BitmapAnd': 18}
{'Actual Total Time': RobustScaler(), 'Plan Rows': RobustScaler(), 'Total Cost': RobustScaler()}


In [5]:
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
model = tm.TanJI(configs)
model_dict = torch.load(
    os.path.join(ROOT_DIR, 'checkpoints', 'TanJI.ckpt')
)
model.load_state_dict(model_dict["state_dict"])

logger = pl.loggers.WandbLogger(project=configs.project_name, name=nowtime+'test')
logger.log_hyperparams(configs.__dict__)
trainer = tm.PLTrainer(
    accelerator=configs.accelerator,
    enable_progress_bar=configs.progress_bar,
    enable_model_summary=configs.progress_bar,
    max_epochs=configs.epochs,
    logger=logger,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanji. Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
# import pickle

# # 存储类的实例
# def save_instance(instance, filename):
#     with open(filename, 'wb') as file:
#         pickle.dump(instance, file)

# # 加载存储的实例
# def load_instance(filename):
#     with open(filename, 'rb') as file:
#         instance = pickle.load(file)
#     return instance
# # for zero-shot data
# files_name = []

# for wk_item in workloads:
#     files_name.append(path.join(ROOT_DIR, 'data', 'workload1', wk_item + '_filted.json'))
# # for i in range(20):
# #     files_name.append(path.join(ROOT_DIR, 'data', 'imdb', 'plan_and_cost', 'train_plan_part{}.csv'.format(i)))

# data_file_name = path.join(ROOT_DIR, 'data', 'workload1', configs.dataset_file_name)
# # data_file_name = path.join(ROOT_DIR, 'data', 'imdb', configs.dataset_file_name)

# if os.path.exists(data_file_name):
#     alldataset = load_instance(data_file_name)
# else:
#     dict_list = tu.Encoder.format_workload(files_name)
#     # dict_list = tu.Encoder.format_imdb(files_name)
#     alldataset = tu.PlanTreeDataSet(dict_list, encoder)
#     save_instance(alldataset, data_file_name)

# train_dataloader, val_dataloader, test_dataloader = tu.get_dataloader(alldataset, configs)


In [7]:
# workload 1

for wk_item in workloads:
    files_name = [path.join(ROOT_DIR, 'data', 'workload1', wk_item + '_filted.json')]

    dict_list = tu.Encoder.format_workload(files_name)
    alldataset = tu.PlanTreeDataSet(dict_list, encoder)

    test_data = []
    for plan_meta in alldataset:
        test_data.append(plan_meta[:-1])        
    bs = configs.batch_size
    test_dataloader = DataLoader(dataset=test_data, batch_size=bs, shuffle=True)

    result = trainer.test(model, dataloaders=test_dataloader, test_data_name=wk_item)

torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Size([10000, 20]) torch.Size([10000, 20])
torch.Size([10000, 1, 420]) torch.Size([10000, 20, 20]) torch.Si

In [9]:
# # workload 3
# test_workloads = ['job-light']
# files_name = []
# for tw in test_workloads:
#     files_name.append(path.join(ROOT_DIR, 'data', 'workload2', tw + '_plans.json'))

# dict_list = tu.Encoder.format_imdb_test(files_name)
# alldataset = tu.PlanTreeDataSet(dict_list, encoder)

# test_data = []
# for plan_meta in alldataset:
#     test_data.append(plan_meta[:-1])        
# bs = configs.batch_size
# test_dataloader = DataLoader(dataset=test_data, batch_size=bs, shuffle=True)




# result = trainer.test(model, dataloaders=test_dataloader)

# 计算模型参数数量
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("模型参数数量:", num_params)

模型参数数量: 72418
